In [4]:
# library

import os
import re
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, SGD



In [9]:
df_smile

,sm_name,#,(,),+,-,/,1,2,3,...,S,[,\,],c,l,n,o,r,s
0,Clotrimazole,0,2,2,0,0,0,8,0,0,...,0,0,0,0,21,1,2,0,0,0
1,Mometasone Furoate,0,8,8,0,0,0,4,2,2,...,0,8,0,8,4,2,0,1,0,0
2,Idelalisib,0,3,3,0,1,0,6,4,0,...,0,2,0,2,19,0,6,0,0,0
3,Vandetanib,0,3,3,0,0,0,4,2,2,...,0,0,0,0,14,0,2,0,1,0
4,Bosutinib,1,6,6,0,0,0,2,2,2,...,0,0,0,0,15,2,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,CGM-097,0,11,11,0,0,0,6,2,2,...,0,3,0,3,18,1,0,0,0,0
142,TGX 221,0,4,4,0,0,0,2,4,2,...,0,0,0,0,14,0,2,0,0,0
143,Azacitidine,0,4,4,0,0,0,2,2,0,...,0,4,0,4,3,0,3,0,0,0
144,Atorvastatin,0,9,9,0,2,0,2,6,0,...,0,2,0,2,22,0,1,0,0,0


In [5]:
# feature select
# feature는 cell_type, sm_name, mean value, SMILES 이용, 
# 최대한 id_map에 맞게 최소한으로 설정할건데 일단은 de_train을 기준으로 해보자

de_train = pd.read_parquet("/home/aiuser/taeuk/open-problems-single-cell-perturbations/de_train.parquet")
id_map = pd.read_csv("/home/aiuser/taeuk/open-problems-single-cell-perturbations/id_map.csv")
submission = pd.read_csv("/home/aiuser/taeuk/open-problems-single-cell-perturbations/sample_submission.csv")

sm_name_id_map = sorted(id_map.sm_name.unique())
cell_type_id_map = sorted(id_map.cell_type.unique())

# cell & compound dictionary

cell_type_de_train = sorted(de_train.cell_type.unique())
sm_name_de_train = sorted(de_train.sm_name.unique())

cell_type_dict = {cell_type_de_train[i]:i for i in range(len(cell_type_de_train))}
sm_name_dict = {sm_name_de_train[i]:i for i in range(len(sm_name_de_train))}

# compound info by SMILES

smiles = list(de_train.SMILES.unique())
voc = []

r = re.compile(".")
for sm in smiles:
    voc += list(set(r.findall(sm)))
voc = list(set(voc))
voc.sort()

df_smile = pd.DataFrame(np.zeros((len(smiles), len(voc))).astype(int))
df_smile.columns = voc
for i in range(df_smile.shape[0]):
    for ele in r.findall(smiles[i]):
        df_smile[ele][i] += 1
df_smile = pd.DataFrame(smiles).join(df_smile)
df_smile.columns = ["SMILES"] + list(df_smile.columns[1:])
sm_id = de_train.iloc[:, 1:4].drop("sm_lincs_id", axis=1).drop_duplicates()
df_smile = sm_id.merge(df_smile, how="left", on="SMILES")
df_smile.drop("SMILES", axis=1, inplace=True)

# build dataloader

class SCPset(Dataset):
    def __init__(self, dataset, df_smile, cell_type_dict, sm_name_dict):
        super(SCPset, self).__init__()
        if dataset is None:
            self.x = id_map.iloc[:, 1:]
            self.y = None
        else:
            self.x = dataset.iloc[:, :2]
            self.y = dataset.iloc[:, 5:]
        self.smile = df_smile
        self.cell_type_dict = cell_type_dict
        self.sm_name_dict = sm_name_dict
   
    def __getitem__(self, idx):
        cell, name = self.x.iloc[idx]
        x_cell = self.cell_type_dict[cell]
        x_comp = self.smile.loc[self.smile.sm_name==name, "#":].values[0]
        ele_idx = np.where(x_comp!=0)[0][1:]
        ele_val = x_comp[ele_idx]
        ele_idx = list(ele_idx) + [0]*(20-len(ele_idx))
        ele_val = list(ele_val) + [0]*(20-len(ele_val))
        x = [x_cell] + list(ele_idx) + list(ele_val)
        if self.y is None:
            return torch.tensor(x, dtype=torch.int64)
        else:
            y = self.y.iloc[idx, :]
            return torch.tensor(x, dtype=torch.int64),\
                    torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return self.x.shape[0]     

In [3]:
# Define model
'''
input : [cell, comp, smil]
x     : tensor size of (B, dim_embed)      
'''
class SCPblock(nn.Module):
    def __init__(self, dim_embed, device):
        super(SCPblock, self).__init__()
        self.fc1 = nn.Linear(dim_embed, dim_embed, device=device)
        self.fc2 = nn.Linear(dim_embed, dim_embed, device=device)
        self.elu = nn.ELU()
        
    def forward(self ,x):
        x_ = self.elu(self.fc1(x))
        return self.elu(self.fc2(x_)+x)
        
class SCPmodel(nn.Module):
    def __init__(self, dim_embed, num_layers, num_adds, gene, device):
        super(SCPmodel, self).__init__()
        self.embed_cell = nn.Embedding(6, dim_embed, device=device)
        self.embed_comp = nn.Embedding(33, dim_embed*2, padding_idx=0, device=device)
        self.embed_gene = torch.randn(size=(1, gene, dim_embed*3),
                                      device=device, requires_grad=True)
        self.num_adds = num_adds
        self.layers = nn.ModuleList([
            SCPblock(dim_embed*3, device) for _ in range(num_layers) ])
            
    def forward(self, x):
        cell = self.embed_cell(x[:, 0]).unsqueeze(1)
        comp_idx = x[:, 1:21]
        comp_val = x[:, 21:]
        comp = self.embed_comp(comp_idx)
        comp = comp * comp_val.unsqueeze(2)
        comp = comp.mean(dim=1, keepdim=True)
        x = torch.cat([cell, comp], dim=2)
        x = x.expand(cell.shape[0], self.embed_gene.shape[1], cell.shape[2]*3)
        x = x + self.embed_gene
        for i in range(len(self.layers)):
            x = self.layers[i](x)
            # # add gene embedding
            # if i % (len(self.layers) // self.num_adds)==0:
            #     x = x + self.embed_gene
        return x.mean(dim=2, keepdim=False)

In [4]:
# Define util function

def MRRMSE(pred, y):
      pred = pred.detach().cpu().numpy()
      y = y.detach().cpu().numpy()
      return np.sqrt(np.square(y - pred).mean(axis=1)).mean()    

def mrrmse_loss(pred, y):
      return torch.sqrt(torch.square(pred - y).mean(dim=1)).mean()

def compose_loss(pred, y):
      return mrrmse_loss(pred, y) + F.smooth_l1_loss(pred, y)

def fix_random_seed(seed):
    #random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [5]:
# Define training function

def train_model(device, train_size, dim_model, num_layers, num_adds, genes, optimizer, criterion, learning_rate, num_epochs, batch_size, verbose):
      
      train_idx = np.random.choice(de_train.shape[0], int(de_train.shape[0]*train_size), replace=False)
      train_loader = DataLoader(SCPset(de_train.iloc[train_idx, :], df_smile, cell_type_dict, sm_name_dict),
                                batch_size=batch_size, shuffle=True)
      valid_loader = [1]
      if train_size < 1.:
            valid_idx = list(set(np.arange(de_train.shape[0])) - set(train_idx))
            valid_loader = DataLoader(SCPset(de_train.iloc[valid_idx, :], df_smile, cell_type_dict, sm_name_dict),
                                      batch_size=batch_size, shuffle=True)

      model = SCPmodel(dim_model, num_layers, num_adds, genes, device)
      
      if optimizer.lower()=="sgd":
            optimizer = SGD(list(model.parameters())+[model.embed_gene], lr=learning_rate, momentum=0.)
      elif optimizer.lower()=='adam':
            optimizer = Adam(list(model.parameters())+[model.embed_gene], lr=learning_rate)
      else:
            print("Wrong optimizer!")
            return
      
      scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs)
      
      if criterion.lower()=="l1":
            criterion = nn.SmoothL1Loss(0.5)
      elif criterion.lower()=="l2":
            criterion = nn.MSELoss()
      elif criterion.lower()=="MRRMSE":
            criterion = mrrmse_loss
      elif criterion.lower() == "compose":
            criterion = compose_loss
      else:
            print("Wrong criterion!")
            return
      
      best_mrrmse = 10
      for epoch in tqdm(range(1, num_epochs+1)):
            train_loss = 0.
            valid_loss = 0.
            train_mrrmse = 0.
            valid_mrrmse = 0.
            
            model.train()
            for x, y in train_loader:
                  x, y = x.to(device), y.to(device)
                  optimizer.zero_grad()
                  pred = model(x)
                  if F.mse_loss(pred[0,:], pred[1,:]).item() < 0.0001:
                        print("trivial solution !, mse of prediction is %.6f"%F.mse_loss(pred[0,:], pred[1,:]).item())
                  loss = criterion(pred, y)
                  model.embed_gene.retain_grad()
                  loss.backward()
                  optimizer.step()
                  
                  train_loss += loss.item()
                  train_mrrmse += MRRMSE(pred, y)
            if train_size < 1.:
                  model.eval()
                  for x, y in valid_loader:
                        x, y = x.to(device), y.to(device)
                        with torch.no_grad():
                              pred = model(x)
                              loss = criterion(pred, y)
                        
                        valid_loss += loss.item()
                        valid_mrrmse += MRRMSE(pred, y)
            scheduler.step()      
            # print loss per epoch
            if verbose:
                  print("[Epoch : %2d] [Loss : %.4f / %.4f] [MRRMSE : %.3f / %.3f]"%(
                        epoch, train_loss/len(train_loader), valid_loss/len(valid_loader),
                        train_mrrmse/len(train_loader), valid_mrrmse/len(valid_loader)))
            if train_size < 1. and best_mrrmse > valid_mrrmse/len(valid_loader):
                  best_mrrmse = valid_mrrmse/len(valid_loader)
            
      return model, best_mrrmse

def infer_model(device, model):
      test_loader = DataLoader(SCPset(None, df_smile, cell_type_dict, sm_name_dict),
                               batch_size=255, shuffle=False)
      model.eval()
      for x in test_loader:
            x = x.to(device)
            with torch.no_grad():
                  pred = model(x)
      return pred

In [6]:
# hyper parameter tuning

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# result = []
# fix_random_seed(231111)
# train_size = 0.7
# num_epochs = 10
# for dim_model in [16]:
#     for num_layers in [14]:
#         for learning_rate in [0.05]:
#             for batch_size in [64]:
#                 model, mrrmse = train_model(device, train_size, dim_model, num_layers, 18211,
#                                         "Adam", "L1", learning_rate, num_epochs, batch_size, True)
#                 res = [dim_model, num_layers, learning_rate, batch_size, mrrmse]
#                 result.append(res)
# result = np.array(result)
# result

# gene_info = model.embed_gene.data.squeeze().cpu().numpy()
# gene_info = pd.DataFrame(gene_info)
# display(gene_info)

In [7]:
# training & prediction

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
      
train_size = 1.
dim_model = 64
num_layers = 1
num_adds = 1 # 많이 더할수록(입력할수록) prediction이 유전자마다 비슷비슷해짐
learning_rate = 0.04
batch_size = 600
num_epochs = 100 # 10 으로 했을때 0.65

fix_random_seed(231111)
model, mrrmse = train_model(device, train_size, dim_model, num_layers, num_adds,
                            18211, "Adam", "L1", learning_rate, num_epochs, batch_size, True)
pred = infer_model(device, model)
data = pred.detach().cpu().numpy()
df = pd.DataFrame(data).reset_index()
df.columns = submission.columns
display(df.head())

gene_info = model.embed_gene.data.squeeze().cpu().numpy()
gene_info = pd.DataFrame(gene_info)
gene_info.index = submission.columns[1:]
cell_info = model.embed_cell.weight.detach().cpu().numpy()
cell_info = pd.DataFrame(cell_info)
cell_info.index = [k for k in cell_type_dict.keys()]
df_corr = cell_info.T.corr()
df_corr.columns = [k for k in cell_type_dict.keys()]
df_corr.index = [k for k in cell_type_dict.keys()]

display(gene_info)
display(cell_info)
display(df_corr)

/home/aiuser/.conda/envs/taeuk/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


  0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipykernel_1876720/2686368559.py:72: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  torch.tensor(y, dtype=torch.float32)


OutOfMemoryError: CUDA out of memory. Tried to allocate 7.82 GiB. GPU 0 has a total capacty of 23.64 GiB of which 7.73 GiB is free. Including non-PyTorch memory, this process has 15.90 GiB memory in use. Of the allocated memory 15.70 GiB is allocated by PyTorch, and 9.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# save

title = f"scp_D{dim_model}_L{num_layers}_A{num_adds}_lr{learning_rate}_B{batch_size}_E{num_epochs}"
df.to_csv("/home/aiuser/taeuk/%s.csv"%title, header=True, index=False)
